In [ ]:
# Notebook: Análisis Exploratorio de Ventas de Motos en Colombia
# ================================================================

# %% [markdown]
# # 🏍️ Análisis Exploratorio de Datos (EDA)
# ## Ventas de Motos en Colombia
# 
# **Objetivos:**
# - Cargar y limpiar el dataset
# - Crear rangos de cilindrada
# - Analizar ventas por ciudad
# - Visualizar distribuciones y tendencias
# - Preparar datos para Machine Learning

# %% [markdown]
# ## 1. Importar Librerías

# %%
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("✅ Librerías importadas correctamente")

# %% [markdown]
# ## 2. Cargar Dataset

# %%
# Cargar datos
df = pd.read_csv('../data/ventas_motos_colombia.csv')

print(f"📊 Dataset cargado: {df.shape[0]} filas x {df.shape[1]} columnas")
print(f"\n🔍 Primeras filas del dataset:")
df.head()

# %% [markdown]
# ## 3. Exploración Inicial

# %%
# Información general del dataset
print("📋 Información del Dataset:")
print(df.info())

print("\n📊 Estadísticas descriptivas:")
df.describe()

# %%
# Verificar valores nulos
print("❓ Valores Nulos por Columna:")
nulos = df.isnull().sum()
print(nulos[nulos > 0])

if nulos.sum() == 0:
    print("✅ No hay valores nulos en el dataset")

# %%
# Verificar duplicados
duplicados = df.duplicated().sum()
print(f"🔁 Registros duplicados: {duplicados}")

if duplicados > 0:
    df = df.drop_duplicates()
    print(f"✅ Duplicados eliminados. Nuevo tamaño: {df.shape}")

# %% [markdown]
# ## 4. Creación de Rangos de Cilindrada

# %%
def crear_rangos_cc(cc):
    """Clasifica las motos por rangos de cilindrada"""
    if pd.isna(cc):
        return 'Desconocido'
    cc = float(cc)
    if cc <= 100:
        return '0-100cc'
    elif cc <= 200:
        return '100-200cc'
    elif cc <= 300:
        return '200-300cc'
    elif cc <= 400:
        return '300-400cc'
    elif cc <= 500:
        return '400-500cc'
    elif cc <= 600:
        return '500-600cc'
    else:
        return '600+cc'

# Aplicar la función
col_cc = 'cilindrada' if 'cilindrada' in df.columns else 'cc'
df['rango_cc'] = df[col_cc].apply(crear_rangos_cc)

print("✅ Rangos de cilindrada creados")
print("\n📊 Distribución de rangos:")
print(df['rango_cc'].value_counts().sort_index())

# %% [markdown]
# ## 5. Análisis de Ventas por Rango de Cilindrada

# %%
# Gráfico de barras: Ventas por rango de CC
plt.figure(figsize=(12, 6))
ventas_cc = df['rango_cc'].value_counts().sort_index()
ventas_cc.plot(kind='bar', color='steelblue', edgecolor='black')
plt.title('Ventas de Motos por Rango de Cilindrada', fontsize=16, fontweight='bold')
plt.xlabel('Rango de Cilindrada', fontsize=12)
plt.ylabel('Cantidad de Ventas', fontsize=12)
plt.xticks(rotation=45)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

# %% [markdown]
# ## 6. Top 5 Ciudades con Más Ventas

# %%
# Obtener top 5 ciudades
if 'ciudad' in df.columns:
    top_ciudades = df['ciudad'].value_counts().head(5)
    
    print("🏆 Top 5 Ciudades con Más Ventas:")
    print("=" * 40)
    for i, (ciudad, ventas) in enumerate(top_ciudades.items(), 1):
        print(f"{i}. {ciudad}: {ventas:,} ventas")
    
    # Gráfico de barras horizontales
    plt.figure(figsize=(10, 6))
    top_ciudades.sort_values().plot(kind='barh', color='coral', edgecolor='black')
    plt.title('Top 5 Ciudades con Más Ventas', fontsize=16, fontweight='bold')
    plt.xlabel('Cantidad de Ventas', fontsize=12)
    plt.ylabel('Ciudad', fontsize=12)
    plt.grid(axis='x', alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Gráfico circular
    plt.figure(figsize=(10, 8))
    plt.pie(top_ciudades, labels=top_ciudades.index, autopct='%1.1f%%',
            startangle=90, colors=sns.color_palette('Set3'))
    plt.title('Distribución de Ventas - Top 5 Ciudades', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ No se encontró columna 'ciudad' en el dataset")

# %% [markdown]
# ## 7. Análisis Cruzado: Ciudad vs Rango de Cilindrada

# %%
if 'ciudad' in df.columns:
    # Crear tabla cruzada
    tabla_cruzada = pd.crosstab(df['ciudad'], df['rango_cc'])
    
    # Filtrar top 10 ciudades para mejor visualización
    top_10_ciudades = df['ciudad'].value_counts().head(10).index
    tabla_cruzada_top = tabla_cruzada.loc[top_10_ciudades]
    
    # Heatmap
    plt.figure(figsize=(14, 8))
    sns.heatmap(tabla_cruzada_top, annot=True, fmt='d', cmap='YlOrRd', 
                linewidths=0.5, cbar_kws={'label': 'Cantidad de Ventas'})
    plt.title('Mapa de Calor: Ventas por Ciudad y Rango de Cilindrada', 
              fontsize=16, fontweight='bold')
    plt.xlabel('Rango de Cilindrada', fontsize=12)
    plt.ylabel('Ciudad', fontsize=12)
    plt.tight_layout()
    plt.show()
    
    print("✅ Análisis cruzado completado")

# %% [markdown]
# ## 8. Análisis de Precios

# %%
if 'precio' in df.columns:
    # Estadísticas de precios por rango de CC
    print("💰 Estadísticas de Precios por Rango de Cilindrada:")
    print("=" * 60)
    precios_por_cc = df.groupby('rango_cc')['precio'].describe()
    print(precios_por_cc)
    
    # Boxplot
    plt.figure(figsize=(14, 6))
    orden = ['0-100cc', '100-200cc', '200-300cc', '300-400cc', '400-500cc', '500-600cc', '600+cc']
    orden_disponibles = [o for o in orden if o in df['rango_cc'].values]
    sns.boxplot(data=df, x='rango_cc', y='precio', order=orden_disponibles, palette='Set2')
    plt.title('Distribución de Precios por Rango de Cilindrada', fontsize=16, fontweight='bold')
    plt.xlabel('Rango de Cilindrada', fontsize=12)
    plt.ylabel('Precio (COP)', fontsize=12)
    plt.xticks(rotation=45)
    plt.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    plt.show()

# %% [markdown]
# ## 9. Correlación entre Variables Numéricas

# %%
# Seleccionar solo columnas numéricas
numeric_cols = df.select_dtypes(include=[np.number]).columns
df_numeric = df[numeric_cols]

# Matriz de correlación
plt.figure(figsize=(10, 8))
correlation_matrix = df_numeric.corr()
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            square=True, linewidths=0.5, cbar_kws={'label': 'Correlación'})
plt.title('Matriz de Correlación - Variables Numéricas', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

print("📊 Variables más correlacionadas:")
print(correlation_matrix.unstack().sort_values(ascending=False).drop_duplicates())

# %% [markdown]
# ## 10. Análisis Temporal (si existe fecha)

# %%
# Buscar columna de fecha
col_fecha = None
for col in ['fecha', 'date', 'año', 'year']:
    if col in df.columns:
        col_fecha = col
        break

if col_fecha:
    try:
        df['fecha_parsed'] = pd.to_datetime(df[col_fecha], errors='coerce')
        df_temporal = df.dropna(subset=['fecha_parsed'])
        
        # Agrupar por mes
        ventas_mes = df_temporal.groupby(df_temporal['fecha_parsed'].dt.to_period('M')).size()
        
        plt.figure(figsize=(14, 6))
        ventas_mes.plot(kind='line', marker='o', color='steelblue', linewidth=2, markersize=6)
        plt.title('Evolución Temporal de Ventas', fontsize=16, fontweight='bold')
        plt.xlabel('Mes', fontsize=12)
        plt.ylabel('Cantidad de Ventas', fontsize=12)
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
        
        print("✅ Análisis temporal completado")
    except Exception as e:
        print(f"⚠️ No se pudo realizar análisis temporal: {e}")
else:
    print("⚠️ No se encontró columna de fecha en el dataset")

# %% [markdown]
# ## 11. Preparación de Datos para Machine Learning

# %%
print("🤖 Preparando datos para Machine Learning...")

# Identificar columnas numéricas
numeric_features = df.select_dtypes(include=[np.number]).columns.tolist()

# Buscar variable objetivo
target_col = None
for col in ['ventas', 'cantidad', 'unidades_vendidas', 'total']:
    if col in numeric_features:
        target_col = col
        break

if target_col and len(numeric_features) > 1:
    # Features y target
    features = [col for col in numeric_features if col != target_col]
    X = df[features].fillna(0)
    y = df[target_col]
    
    print(f"✅ Variable objetivo: {target_col}")
    print(f"✅ Features seleccionadas: {features}")
    print(f"✅ Shape de X: {X.shape}")
    print(f"✅ Shape de y: {y.shape}")
    
    # Split train/test
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    
    print(f"\n📊 Split realizado:")
    print(f"  - Train: {X_train.shape[0]} muestras")
    print(f"  - Test: {X_test.shape[0]} muestras")
else:
    print("⚠️ No se pudieron identificar features suficientes para ML")

# %% [markdown]
# ## 12. Entrenamiento de Modelo Básico

# %%
if target_col and len(numeric_features) > 1:
    # Entrenar modelo de Regresión Lineal
    modelo = LinearRegression()
    modelo.fit(X_train, y_train)
    
    # Predicciones
    y_pred = modelo.predict(X_test)
    
    # Métricas
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    
    print("📈 Resultados del Modelo de Regresión Lineal:")
    print("=" * 50)
    print(f"  R² Score: {r2:.4f}")
    print(f"  MAE: {mae:.2f}")
    print(f"  RMSE: {rmse:.2f}")
    
    # Gráfico de predicciones vs reales
    plt.figure(figsize=(10, 6))
    plt.scatter(y_test, y_pred, alpha=0.6, color='steelblue')
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 
             'r--', lw=2, label='Ideal')
    plt.xlabel('Valores Reales', fontsize=12)
    plt.ylabel('Predicciones', fontsize=12)
    plt.title('Predicción vs Valores Reales', fontsize=16, fontweight='bold')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Guardar modelo
    import pickle
    with open('../models/modelo_regresion.pkl', 'wb') as f:
        pickle.dump(modelo, f)
    
    print("\n✅ Modelo guardado en: ../models/modelo_regresion.pkl")

# %% [markdown]
# ## 13. Resumen de Insights

# %%
print("📊 RESUMEN DE INSIGHTS CLAVE")
print("=" * 60)

if 'rango_cc' in df.columns:
    rango_mas_vendido = df['rango_cc'].value_counts().index[0]
    print(f"🏍️ Rango de cilindrada más vendido: {rango_mas_vendido}")

if 'ciudad' in df.columns:
    ciudad_top = df['ciudad'].value_counts().index[0]
    ventas_top = df['ciudad'].value_counts().iloc[0]
    print(f"🏆 Ciudad con más ventas: {ciudad_top} ({ventas_top:,} ventas)")

if 'precio' in df.columns:
    precio_promedio = df['precio'].mean()
    print(f"💰 Precio promedio: ${precio_promedio:,.0f} COP")

print(f"📈 Total de registros analizados: {len(df):,}")

if target_col and 'r2' in locals():
    print(f"🤖 R² del modelo predictivo: {r2:.4f}")

print("\n✅ Análisis exploratorio completado exitosamente!")

# %%